In [23]:
import numpy as np
import EFIE as solve
light_speed, mu0, eps0 = 299792458., 4*np.pi*1e-7, 8.854e-12

frequency = 1e6 * 146
omega, wavelength = 2 * np.pi * frequency, light_speed / frequency
incident_field, radius = 220, 3.175e-3
delta_r = wavelength / 100

class yagi:
    def __init__(self, position, angle, length, source_position, radius):
        self.position = position
        self.angle = angle
        self.length = length
        self.source_position = source_position
        self.radius = radius

class tree:
    def __init__(self, its, phi, length, f, radius, field):
        self.its = its
        self.phi = phi
        self.length = length
        self.f = f
        self.radius = radius
        self.field = field

basis_functions = 'triangle'
structure_type = 'yagi-uda'
position = np.array([[0.0, 0.0, 0.0]])
angle = np.array([[np.pi/2, 0.0]])
length = np.array([delta_r*5])
source_position = np.array([[0.0, 0.0, 0.0, incident_field]])
radii = np.array([radius])
test_yagi = yagi(position, angle, length, source_position, radii)

In [24]:
c, mu0, eps0 = 299792458., 4*np.pi*1e-7, 8.854e-12

def basis_func(basis_functions, t_n, r_n, dr_n):
    if basis_functions == 'pulse' :
        return 1
    elif basis_functions == 'triangle':
        if -1/2 <= t_n <= 1/2 :
            return 1/2 + t_n
        elif 1/2 <= t_n <= 3/2 :
            return 3/2 - t_n

def weight_func(basis_functions, t_m, r_m, dr_m):
    if basis_functions == 'pulse' :
            return 1
    elif basis_functions == 'triangle':
            if -1/2 <= t_m <= 1/2 :
                return 1/2 + t_m
            elif 1/2 <= t_m <= 3/2 :
                return 3/2 - t_m
            
def impedance_real (t_m, t_n, r_m, r_n, dr_m, dr_n, omega, basis_functions):
    k = omega / c
    
    tau_n = dr_n / np.linalg.norm(dr_n)
    tau_m = dr_m / np.linalg.norm(dr_m)
    
    c_0 = np.dot(tau_m, tau_n)
    c_x = 1/k**2 * tau_n[0] * tau_m[0]
    c_y = 1/k**2 * tau_n[1] * tau_m[1]
    c_z = 1/k**2 * tau_n[2] * tau_m[2]
    c_xy = 1/k**2 * (tau_m[0]*tau_n[1] + tau_m[1]*tau_n[0])
    c_xz = 1/k**2 * (tau_m[0]*tau_n[2] + tau_m[2]*tau_n[0])
    c_yz = 1/k**2 * (tau_m[1]*tau_n[2] + tau_m[2]*tau_n[1])
    
    sum_c = c_x + c_y + c_z + c_xy + c_xz + c_yz
    
    rmn = np.linalg.norm(r_m - r_n + dr_m*(t_m-1/2) - dr_n*(t_n-1/2))
    dx = r_m[0] - r_n[0] + dr_m[0]*(t_m-1/2) - dr_n[0]*(t_n-1/2)
    dy = r_m[1] - r_n[1] + dr_m[1]*(t_m-1/2) - dr_n[1]*(t_n-1/2)
    dz = r_m[2] - r_n[2] + dr_m[2]*(t_m-1/2) - dr_n[2]*(t_n-1/2)
    psi = c_x * dx**2 + c_y * dy**2 + c_z * dz**2 + c_xy * dx * dy + c_xz * dx * dz + c_yz * dy * dz
    
    polypart1 = c_0 * rmn**2 - sum_c - psi * (k**2 * rmn**2 + 1) / rmn**2
    polypart2 = -k * (rmn * sum_c - psi/rmn)
    
    return polypart1 * np.cos(k * rmn) / rmn**3 + polypart2 * np.sin(k * rmn) / rmn**3
    
def impedance_imag (t_m, t_n, r_m, r_n, dr_m, dr_n, omega, basis_functions):
    k = omega / c
    
    tau_n = dr_n / np.linalg.norm(dr_n)
    tau_m = dr_m / np.linalg.norm(dr_m)
    
    c_0 = np.dot(tau_m, tau_n)
    c_x = 1/k**2 * tau_n[0] * tau_m[0]
    c_y = 1/k**2 * tau_n[1] * tau_m[1]
    c_z = 1/k**2 * tau_n[2] * tau_m[2]
    c_xy = 1/k**2 * (tau_m[0]*tau_n[1] + tau_m[1]*tau_n[0])
    c_xz = 1/k**2 * (tau_m[0]*tau_n[2] + tau_m[2]*tau_n[0])
    c_yz = 1/k**2 * (tau_m[1]*tau_n[2] + tau_m[2]*tau_n[1])
    
    sum_c = c_x + c_y + c_z + c_xy + c_xz + c_yz
    
    rmn = np.linalg.norm(r_m - r_n + dr_m*(t_m-1/2) - dr_n*(t_n-1/2))
    dx = r_m[0] - r_n[0] + dr_m[0]*(t_m-1/2) - dr_n[0]*(t_n-1/2)
    dy = r_m[1] - r_n[1] + dr_m[1]*(t_m-1/2) - dr_n[1]*(t_n-1/2)
    dz = r_m[2] - r_n[2] + dr_m[2]*(t_m-1/2) - dr_n[2]*(t_n-1/2)
    psi = c_x * dx**2 + c_y * dy**2 + c_z * dz**2 + c_xy * dx * dy + c_xz * dx * dz + c_yz * dy * dz
    
    polypart1 = c_0 * rmn**2 - sum_c - psi * (k**2 * rmn**2 + 1) / rmn**2
    polypart2 = -k * (rmn * sum_c - psi/rmn)
    
    return -polypart1 * np.sin(k * rmn) / rmn**3 + polypart2 * np.cos(k * rmn) / rmn**3



In [25]:
import geometry as gm
import matrix_elements as matrix_elements

segments_block, source_segments = gm.antenna_to_segments(structure_type=structure_type, antenna=test_yagi, basis_functions=basis_functions, delta_r=delta_r)
impedance_past = matrix_elements.calculate_impedance(basis_functions=basis_functions, structure_type=structure_type, segments_block=segments_block, frequency=frequency, delta_r=delta_r)
print(np.abs(impedance_past))

[[2072.84425257  697.72763311  257.43335819   46.04172576]
 [ 697.72763311 2072.84425257  697.72763311  257.43335819]
 [ 257.43335819  697.72763311 2072.84425257  697.72763311]
 [  46.04172576  257.43335819  697.72763311 2072.84425257]]


In [26]:
import numpy as np 
from scipy import linalg 
import scipy.integrate as integrate 
from tqdm import tqdm

def calculate_impedance_opt (basis_functions, structure_type, segments_block, frequency, delta_r):
    
    omega = 2 * np.pi * frequency
    
    element_num = []
    for i in range (len(segments_block)):
        element_num.append(len(segments_block[i]))
    element_num = np.array(element_num)
    
    impedance_block = []
    
    if structure_type == 'yagi-uda':    
        for m in range(len(segments_block)):
            impedance_row = []
            for n in range(len(segments_block)):
                impedance_mn = np.zeros((len(segments_block[m]), len(segments_block[n])), dtype=complex)
                if m <= n :
                    for i in range (len(segments_block[m]) + len(segments_block[n])):
                        impedance_mn[max(0, len(segments_block[m])-i-1), max(0, i-len(segments_block[m]))] = Zmn(structure_type=structure_type,basis_functions=basis_functions, m=m, n=n, i=max(0, len(segments_block[m])-i-1), j=max(0, i-len(segments_block[m])), segments_block=segments_block, omega=2*np.pi*frequency, delta_r=delta_r)
                        for k in range (min( min(len(segments_block[m]), len(segments_block[n])), i+1, len(segments_block[m]) + len(segments_block[n]) - i)):
                            impedance_mn[max(0, len(segments_block[m])-i-1) + k, max(0, i-len(segments_block[m])) + k] = impedance_mn[max(0, len(segments_block[m])-i-1), max(0, i-len(segments_block[m]))]
                else :
                    impedance_mn = impedance_block[n][m].T
                impedance_row.append(impedance_mn)   
            impedance_block.append(impedance_row)
    
    num_elements = sum(element_num)
    impedance = np.zeros((num_elements, num_elements), dtype = complex)
    cum_n = np.append(0, np.cumsum(element_num))
    for i in range (len(cum_n)-1):
        for j in range (len(cum_n)-1):
            impedance[cum_n[i]:cum_n[i+1], cum_n[j]:cum_n[j+1]] = impedance_block[i][j]
    return impedance

def Zmn (structure_type, basis_functions, m, n, i, j, segments_block, omega, delta_r):

    if basis_functions == 'pulse' :
        t_min, t_max = 0, 1
    elif basis_functions == 'triangle' :
        t_min, t_max = -1/2, 3/2
        
    a_m = segments_block[m][i].radius
    a_n = segments_block[n][j]. radius
    
    tau_m = segments_block[m][i].tau
    tau_n = segments_block[n][j].tau
    
    r_m = segments_block[m][i].position 
    r_n = segments_block[n][j].position 
    
    if structure_type == 'yagi-uda':
        r_n = r_n + a_n * np.array([0,0,1])
    elif structure_type == 'tree':
        r_n = r_n + a_n * np.array([0,0,1])

    dr_m = delta_r * tau_m
    dr_n = delta_r * tau_n
    
    return 1j*omega*mu0 / (4*np.pi) * delta_r **2 * (integrate.dblquad(impedance_real, t_min, t_max, lambda z1: t_min, lambda z2: t_max, args=(r_m, r_n, dr_m, dr_n, omega, basis_functions))[0] + 1j * integrate.dblquad(impedance_imag, t_min, t_max, lambda z1: t_min, lambda z2: t_max, args=(r_m, r_n, dr_m, dr_n, omega, basis_functions))[0])

In [27]:
impedance = calculate_impedance_opt(basis_functions='triangle', structure_type=structure_type, segments_block=segments_block, frequency=frequency, delta_r=delta_r)

In [ ]:
print(np.abs(impedance))

[[96615.76011144 52896.00387642  4783.31284927   250.66682881]
 [52896.00387642 96615.76011144 52896.00387642  4783.31284927]
 [ 4783.31284927 52896.00387642 96615.76011144 52896.00387642]
 [  250.66682881  4783.31284927 52896.00387642 96615.76011144]]


In [30]:
print(np.abs(impedance)/np.abs(impedance_past))

[[46.61023615 75.81182308 18.58078099  5.44434042]
 [75.81182308 46.61023615 75.81182308 18.58078099]
 [18.58078099 75.81182308 46.61023615 75.81182308]
 [ 5.44434042 18.58078099 75.81182308 46.61023615]]
